In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!python --version

Python 3.7.9


In [3]:
from scipy.optimize import linprog
import numpy as np

# 예시: 생산 계획 문제
# 두 가지 제품 A, B를 생산. 이익을 최대화하는 생산량 결정.
# 제품 A: 생산 시간 1시간, 재료 2단위, 이익 3만원
# 제품 B: 생산 시간 2시간, 재료 1단위, 이익 2만원
# 제약 조건: 총 생산 시간 8시간, 총 재료 6단위

# 목적 함수 계수 (최대화 문제이므로 -1을 곱함)
# -3 * x_A - 2 * x_B 를 최소화
c = np.array([-3, -2])

# 부등식 제약 조건 (Ax <= b)
# 생산 시간: 1*x_A + 2*x_B <= 8
# 재료: 2*x_A + 1*x_B <= 6
A = np.array([[1, 2], [2, 1]])
b = np.array([8, 6])

# 변수 범위 (x_A >= 0, x_B >= 0)
x0_bounds = (0, None) # x_A >= 0
x1_bounds = (0, None) # x_B >= 0

# 선형 계획법 풀이
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method='highs')

print("--- 선형 계획법 결과 ---")
if res.success:
    print(f"최적 생산량 (제품 A, B): {res.x}")
    print(f"최대 이익: {-res.fun:.2f}만원") # -1을 곱했으므로 다시 -1을 곱함
else:
    print("최적해를 찾지 못했습니다.")
print(f"상태: {res.message}")
'''
최적 생산량 (제품 A, B): [1.33333333 3.33333333]
최대 이익: 10.67만원
상태: Optimization terminated successfully.
'''

--- 선형 계획법 결과 ---
최적 생산량 (제품 A, B): [1.33333333 3.33333333]
최대 이익: 10.67만원
상태: Optimization terminated successfully.


'\n최적 생산량 (제품 A, B): [1.33333333 3.33333333]\n최대 이익: 10.67만원\n상태: Optimization terminated successfully.\n'

In [4]:
from pulp import *

# 예시: 투자 선택 문제
# 3가지 프로젝트에 투자할지 말지 결정. 총 예산 1000만원.
# 프로젝트 1: 비용 400만원, 수익 500만원
# 프로젝트 2: 비용 600만원, 수익 700만원
# 프로젝트 3: 비용 300만원, 수익 350만원

# 문제 정의 (최대화 문제)
prob = LpProblem("Investment Problem", LpMaximize)

# 결정 변수 정의 (이진 변수: 0 또는 1)
# x1, x2, x3는 각 프로젝트에 투자할지 말지를 나타내는 이진 변수
x1 = LpVariable("x1", 0, 1, LpBinary)
x2 = LpVariable("x2", 0, 1, LpBinary)
x3 = LpVariable("x3", 0, 1, LpBinary)

# 목적 함수 (총 수익 최대화)
prob += 500 * x1 + 700 * x2 + 350 * x3, "Total Profit"

# 제약 조건 (총 예산 1000만원)
prob += 400 * x1 + 600 * x2 + 300 * x3 <= 1000, "Budget Constraint"

# 문제 풀이
prob.solve()

print("--- 정수 선형 계획법 결과 ---")
print(f"상태: {LpStatus[prob.status]}")

if LpStatus[prob.status] == "Optimal":
    print(f"최적 투자 결정:")
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")
    print(f"최대 총 수익: {value(prob.objective)}만원")
else:
    print("최적해를 찾지 못했습니다.")

E:\ADP-study\.venv\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


--- 정수 선형 계획법 결과 ---
상태: Optimal
최적 투자 결정:
x1 = 1.0
x2 = 1.0
x3 = 0.0
최대 총 수익: 1200.0만원


In [5]:
from pulp import *

# 예시: 생산 및 판매 계획 문제
# 제품 A, B를 생산하여 판매. 제품 A는 연속적으로 생산 가능, 제품 B는 묶음 단위(정수)로만 생산 가능.
# 제품 A: 생산 시간 1시간, 재료 2단위, 이익 3만원
# 제품 B: 생산 시간 2시간, 재료 1단위, 이익 2만원
# 제약 조건: 총 생산 시간 8시간, 총 재료 6단위

# 문제 정의 (최대화 문제)
prob = LpProblem("Mixed Production Problem", LpMaximize)

# 결정 변수 정의
x_A = LpVariable("x_A", 0, None, LpContinuous) # 제품 A 생산량 (연속)
x_B = LpVariable("x_B", 0, None, LpInteger)   # 제품 B 생산량 (정수)

# 목적 함수 (총 이익 최대화)
prob += 3 * x_A + 2 * x_B, "Total Profit"

# 제약 조건
prob += 1 * x_A + 2 * x_B <= 8, "Time Constraint" # 생산 시간
prob += 2 * x_A + 1 * x_B <= 6, "Material Constraint" # 재료

# 문제 풀이
prob.solve()

print("--- 혼합 정수 선형 계획법 결과 ---")
print(f"상태: {LpStatus[prob.status]}")

if LpStatus[prob.status] == "Optimal":
    print(f"최적 생산량:")
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")
    print(f"최대 총 이익: {value(prob.objective)}만원")
else:
    print("최적해를 찾지 못했습니다.")

--- 혼합 정수 선형 계획법 결과 ---
상태: Optimal
최적 생산량:
x_A = 1.5
x_B = 3.0
최대 총 이익: 10.5만원


In [6]:
import torch
import torch.nn as nn
from torchvision import models

# 1. 활성화 함수
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
tanh = nn.Tanh()
softmax = nn.Softmax(dim=1)

# 2. 손실 함수
mse_loss = nn.MSELoss()
mae_loss = nn.L1Loss()
binary_cross_entropy = nn.BCELoss()
categorical_cross_entropy = nn.CrossEntropyLoss() # Softmax 포함

# 3. 옵티마이저
# model.parameters()는 예시이며, 실제 모델의 파라미터를 전달해야 합니다.
# model = YourModel()
# optimizer_sgd = torch.optim.SGD(model.parameters(), lr=0.01)
# optimizer_adam = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer_rmsprop = torch.optim.RMSprop(model.parameters(), lr=0.001)

# 4. 규제 (옵티마이저에 weight_decay 파라미터로 L2 규제 적용)
# optimizer_adam_l2 = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# 5. Dropout
dropout = nn.Dropout(p=0.5)

# 6. 파인튜닝 (예: ResNet18 모델 로드 및 수정)
# 사전 학습된 ResNet18 모델 로드
model_ft = models.resnet18(pretrained=True)

# 모든 파라미터를 고정 (가중치 동결)
for param in model_ft.parameters():
    param.requires_grad = False

# 마지막 분류기(fc layer)를 새로운 작업에 맞게 교체
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10) # 10개 클래스로 분류하는 작업으로 가정

# 교체한 레이어의 파라미터만 학습하도록 설정
# optimizer_ft = torch.optim.Adam(model_ft.fc.parameters(), lr=0.001)

E:\ADP-study\.venv\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
E:\ADP-study\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\kangs/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50

# 1. 활성화 함수
relu = layers.Activation('relu')
sigmoid = layers.Activation('sigmoid')
tanh = layers.Activation('tanh')
softmax = layers.Activation('softmax')

# 2. 손실 함수 (compile 시 문자열로 지정)
# model.compile(loss='mean_squared_error')
# model.compile(loss='mean_absolute_error')
# model.compile(loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy')
# model.compile(loss='sparse_categorical_crossentropy')

# 3. 옵티마이저 (compile 시 문자열 또는 객체로 지정)
# model.compile(optimizer='sgd')
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
# model.compile(optimizer='rmsprop')

# 4. 규제 (레이어에 kernel_regularizer 인자로 L1/L2 규제 적용)
l1_reg = regularizers.l1(0.01)
l2_reg = regularizers.l2(0.01)
# dense_layer = layers.Dense(64, activation='relu', kernel_regularizer=l2_reg)

# 5. Dropout
dropout_layer = layers.Dropout(0.5)

# 6. 파인튜닝 (예: ResNet50 모델 로드 및 수정)
# 사전 학습된 ResNet50 모델 로드 (include_top=False로 분류기 제외)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 베이스 모델의 가중치 동결
base_model.trainable = False

# 새로운 분류기 추가
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation='softmax')(x) # 10개 클래스로 분류
model_ft = keras.Model(inputs, outputs)

# 새로운 분류기만 학습하도록 컴파일
# model_ft.compile(optimizer=keras.optimizers.Adam(),
#                  loss='categorical_crossentropy',
#                  metrics=['accuracy'])

# (선택적) 일부 레이어의 동결 해제 후 미세 조정
# base_model.trainable = True
# for layer in base_model.layers[:-10]: # 마지막 10개 레이어를 제외하고 동결
#     layer.trainable = False
#
# model_ft.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), # 낮은 학습률 사용
#                  loss='categorical_crossentropy',
#                  metrics=['accuracy'])

E:\ADP-study\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\ADP-study\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\ADP-study\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\ADP-study\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' 

Instructions for updating:
Colocations handled automatically by placer.


E:\ADP-study\.venv\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_regression

# 1. 데이터 준비 (분류 예시)
X_clf, y_clf = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_clf_train_scaled = scaler.fit_transform(X_clf_train)
X_clf_test_scaled = scaler.transform(X_clf_test)

# 2. DNN 모델 구축 (이중 분류)
model_dnn_keras = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_clf_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') # 이중 분류
])

# 3. 모델 컴파일 및 학습
model_dnn_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_dnn_keras.fit(X_clf_train_scaled, y_clf_train, epochs=10, batch_size=32, verbose=0)
loss, accuracy = model_dnn_keras.evaluate(X_clf_test_scaled, y_clf_test, verbose=0)
print(f"Keras DNN 이중 분류 정확도: {accuracy:.4f}") # 0.9300

# 4. 데이터 준비 (회귀 예시)
X_reg, y_reg = make_regression(n_samples=1000, n_features=20, n_informative=10, random_state=42)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
scaler_reg_X = StandardScaler()
X_reg_train_scaled = scaler_reg_X.fit_transform(X_reg_train)
X_reg_test_scaled = scaler_reg_X.transform(X_reg_test)
scaler_reg_y = StandardScaler()
y_reg_train_scaled = scaler_reg_y.fit_transform(y_reg_train.reshape(-1, 1))
y_reg_test_scaled = scaler_reg_y.transform(y_reg_test.reshape(-1, 1))

# 5. DNN 모델 구축 (회귀)
model_dnn_reg_keras = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_reg_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear') # 회귀
])

# 6. 모델 컴파일 및 학습
model_dnn_reg_keras.compile(optimizer='adam', loss='mse')
model_dnn_reg_keras.fit(X_reg_train_scaled, y_reg_train_scaled, epochs=10, batch_size=32, verbose=0)
loss_reg = model_dnn_reg_keras.evaluate(X_reg_test_scaled, y_reg_test_scaled, verbose=0)
print(f"Keras DNN 회귀 MSE: {loss_reg:.4f}") # 0.0315

Instructions for updating:
Use tf.cast instead.
Keras DNN 이중 분류 정확도: 0.9350
Instructions for updating:
Use tf.cast instead.
Keras DNN 회귀 MSE: 0.0402


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_regression
from torch.utils.data import DataLoader, TensorDataset

# 1. 데이터 준비 (분류 예시)
X_clf, y_clf = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_clf_train_scaled = scaler.fit_transform(X_clf_train)
X_clf_test_scaled = scaler.transform(X_clf_test)

X_clf_train_tensor = torch.tensor(X_clf_train_scaled, dtype=torch.float32)
y_clf_train_tensor = torch.tensor(y_clf_train, dtype=torch.float32).reshape(-1, 1)
X_clf_test_tensor = torch.tensor(X_clf_test_scaled, dtype=torch.float32)
y_clf_test_tensor = torch.tensor(y_clf_test, dtype=torch.float32).reshape(-1, 1)

train_dataset_clf = TensorDataset(X_clf_train_tensor, y_clf_train_tensor)
train_loader_clf = DataLoader(train_dataset_clf, batch_size=32, shuffle=True)

# 2. DNN 모델 구축 (이중 분류)
class SimpleDNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleDNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid() # 이중 분류

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

model_dnn_pytorch = SimpleDNN(X_clf_train_scaled.shape[1])

# 3. 손실 함수 및 옵티마이저 설정
criterion_clf = nn.BCELoss() # 이진 교차 엔트로피
optimizer_clf = optim.Adam(model_dnn_pytorch.parameters(), lr=0.001)

# 4. 모델 학습
for epoch in range(10):
    for inputs, labels in train_loader_clf:
        optimizer_clf.zero_grad()
        outputs = model_dnn_pytorch(inputs)
        loss = criterion_clf(outputs, labels)
        loss.backward()
        optimizer_clf.step()

# 5. 모델 평가
with torch.no_grad():
    y_pred_proba = model_dnn_pytorch(X_clf_test_tensor)
    y_pred = (y_pred_proba >= 0.5).float()
    accuracy = (y_pred == y_clf_test_tensor).float().mean()
    print(f"PyTorch DNN 이중 분류 정확도: {accuracy.item():.4f}")

PyTorch DNN 이중 분류 정확도: 0.9200


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 1. 데이터 로드 및 전처리
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 2. CNN 모델 구축
model_cnn_keras = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # 10개 클래스 분류
])

# 3. 모델 컴파일 및 학습
model_cnn_keras.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn_keras.fit(train_images, train_labels, epochs=5, batch_size=64, verbose=0)
loss, accuracy = model_cnn_keras.evaluate(test_images, test_labels, verbose=0)
print(f"Keras CNN 분류 정확도: {accuracy:.4f}") # 0.9905

11493376/11490434 [==============================] - 1s 0us/step
Keras CNN 분류 정확도: 0.9893


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. 데이터 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. CNN 모델 구축
class SimpleCNN_PyTorch(nn.Module):
    def __init__(self):
        super(SimpleCNN_PyTorch, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 10) # MNIST 이미지 크기 28x28 -> 풀링 2번 후 7x7

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7) # Flatten
        x = self.fc1(x)
        return x

model_cnn_pytorch = SimpleCNN_PyTorch()

# 3. 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_cnn_pytorch.parameters(), lr=0.001)

# 4. 모델 학습
for epoch in range(5):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_cnn_pytorch(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 5. 모델 평가
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model_cnn_pytorch(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'PyTorch CNN 분류 정확도: {100 * correct / total:.2f}%')

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

PyTorch CNN 분류 정확도: 98.95%


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# 1. 데이터 준비 (간단한 시퀀스 예측 예시)
# 시퀀스 데이터: [0, 1, 2, 3] -> 4 예측
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(data[i + seq_length])
    return np.array(xs), np.array(ys)

data = np.arange(0, 100)
seq_length = 10
X_seq, y_seq = create_sequences(data, seq_length)

X_seq = X_seq.reshape(-1, seq_length, 1) # (samples, timesteps, features)

# 2. RNN 모델 구축
model_rnn_keras = models.Sequential([
    layers.SimpleRNN(32, activation='relu', input_shape=(seq_length, 1)),
    layers.Dense(1) # 회귀 예측
])

# 3. 모델 컴파일 및 학습
model_rnn_keras.compile(optimizer='adam', loss='mse')
model_rnn_keras.fit(X_seq, y_seq, epochs=10, verbose=0)

# 4. 예측
new_sequence = np.array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]).reshape(1, seq_length, 1)
predicted_value = model_rnn_keras.predict(new_sequence)[0][0]
print(f"Keras RNN 예측: {predicted_value:.2f}")

Keras RNN 예측: 102.41


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 1. 데이터 준비 (간단한 시퀀스 예측 예시)
def create_sequences_torch(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(data[i + seq_length])
    return torch.tensor(np.array(xs), dtype=torch.float32).unsqueeze(-1), \
           torch.tensor(np.array(ys), dtype=torch.float32).unsqueeze(-1)

data = np.arange(0, 100)
seq_length = 10
X_seq_torch, y_seq_torch = create_sequences_torch(data, seq_length)

# 2. RNN 모델 구축
class SimpleRNN_PyTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN_PyTorch, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) # 마지막 시점의 은닉 상태 사용
        return out

model_rnn_pytorch = SimpleRNN_PyTorch(input_size=1, hidden_size=32, output_size=1)

# 3. 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model_rnn_pytorch.parameters(), lr=0.01)

# 4. 모델 학습
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model_rnn_pytorch(X_seq_torch)
    loss = criterion(outputs, y_seq_torch)
    loss.backward()
    optimizer.step()

# 5. 예측
new_sequence_torch = torch.tensor(np.array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), dtype=torch.float32).reshape(1, seq_length, 1)
with torch.no_grad():
    predicted_value = model_rnn_pytorch(new_sequence_torch).item()
print(f"PyTorch RNN 예측: {predicted_value:.2f}")

PyTorch RNN 예측: 4.43


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# 1. 데이터 준비 (RNN 예시와 동일)
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(data[i + seq_length])
    return np.array(xs), np.array(ys)

data = np.arange(0, 100)
seq_length = 10
X_seq, y_seq = create_sequences(data, seq_length)

X_seq = X_seq.reshape(-1, seq_length, 1) # (samples, timesteps, features)

# 2. LSTM 모델 구축
model_lstm_keras = models.Sequential([
    layers.LSTM(32, activation='relu', input_shape=(seq_length, 1)),
    layers.Dense(1) # 회귀 예측
])

# 3. 모델 컴파일 및 학습
model_lstm_keras.compile(optimizer='adam', loss='mse')
model_lstm_keras.fit(X_seq, y_seq, epochs=10, verbose=0)

# 4. 예측
new_sequence = np.array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]).reshape(1, seq_length, 1)
predicted_value = model_lstm_keras.predict(new_sequence)[0][0]
print(f"Keras LSTM 예측: {predicted_value:.2f}")

Keras LSTM 예측: 104.18


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 1. 데이터 준비 (RNN 예시와 동일)
def create_sequences_torch(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(data[i + seq_length])
    return torch.tensor(np.array(xs), dtype=torch.float32).unsqueeze(-1), \
           torch.tensor(np.array(ys), dtype=torch.float32).unsqueeze(-1)

data = np.arange(0, 100)
seq_length = 10
X_seq_torch, y_seq_torch = create_sequences_torch(data, seq_length)

# 2. LSTM 모델 구축
class SimpleLSTM_PyTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleLSTM_PyTorch, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :]) # 마지막 시점의 은닉 상태 사용
        return out

model_lstm_pytorch = SimpleLSTM_PyTorch(input_size=1, hidden_size=32, output_size=1)

# 3. 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model_lstm_pytorch.parameters(), lr=0.01)

# 4. 모델 학습
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model_lstm_pytorch(X_seq_torch)
    loss = criterion(outputs, y_seq_torch)
    loss.backward()
    optimizer.step()

# 5. 예측
new_sequence_torch = torch.tensor(np.array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), dtype=torch.float32).reshape(1, seq_length, 1)
with torch.no_grad():
    predicted_value = model_lstm_pytorch(new_sequence_torch).item()
print(f"PyTorch LSTM 예측: {predicted_value:.2f}")

PyTorch LSTM 예측: 1.91
